In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
import ast
import os


fig_dir = './figs/'
os.makedirs(fig_dir, exist_ok=True)

df_all = pd.read_csv('paper-figure_celllist_all.csv')
print(f'Number of samples: {df_all.shape[0]}')

custom_stack_order = ['Squ.epi', 'Squ.meta', 'Navicular', 'Para.Squ', 'Para.Clust', 'Glan', 'Leu','Debris', 'LSIL', 'HSIL', 'Adenocarcinoma']
legend_stack_order = ['Leu', 'Squ.epi', 'Navicular', 'Para.Squ', 'Squ.meta', 'Glan', 'Para.Clust', 'LSIL', 'HSIL', 'Adenocarcinoma', 'Debris']
legend_labels= ['Leukocyte', 'Superficial/intermediate cell', 'Navicular cell', 'Parabasal cell', 'Squamous metaplasia', 'Glandular cell', 'Miscellaneous cell cluster', 'LSIL', 'HSIL', 'Adenocarcinoma', 'Irrelevant object']
CLASS_ORDER = ["NILM", "ASC-US", "LSIL", "ASC-H", "HSIL", "SCC", "AGC"]


probability_threshold = 0.8
for param in custom_stack_order:
    df_all[param] = df_all[f'class_count-{probability_threshold}'].apply(ast.literal_eval).apply(lambda x: x.get(param, 0))

Number of samples: 1124


In [2]:
def summarize_counts(df, metric_col):
    def _agg(g):
        v = g[metric_col].dropna().astype(float).values
        if v.size == 0:
            return pd.Series({"n":0,"median":np.nan,"q1":np.nan,"q3":np.nan,
                              "mean":np.nan,"sd":np.nan,"min":np.nan,"max":np.nan})
        return pd.Series({
            "n": v.size,
            "median": np.median(v),
            "q1": np.percentile(v, 25),
            "q3": np.percentile(v, 75),
            "mean": v.mean(),
            "sd": v.std(ddof=1) if v.size > 1 else 0.0,
            "min": v.min(),
            "max": v.max()
        })
    return (df.groupby(["Facility","Class"], dropna=False)
              .apply(_agg).reset_index())

def bh_correction_vector(p):
    """
    Return Benjamini-Hochberg adjusted q-values for a 1D array of p-values (keeping NaNs).
    """
    p = np.asarray(p, dtype=float)
    q = np.full_like(p, np.nan, dtype=float)

    mask = ~np.isnan(p)
    m = mask.sum()
    if m == 0:
        return q

    idx = np.flatnonzero(mask)    
    p_masked = p[idx]
    order = np.argsort(p_masked, kind="mergesort")
    ranked = p_masked[order] * (m / np.arange(1, m + 1))
    ranked[::-1] = np.minimum.accumulate(ranked[::-1]) 
    q_masked = np.clip(ranked, 0, 1)

    q[idx[order]] = q_masked
    return q


def tests_vs_nilm_one_sided_with_bh(df, metric_col):
    rows = []
    for fac, g in df.groupby("Facility", dropna=False):
        if "NILM" not in g["Class"].unique():
            continue
        x = g.loc[g["Class"]=="NILM", metric_col].dropna().astype(float).values
        comps = [c for c in g["Class"].unique() if c != "NILM"]
        tmp, pvals = [], []
        for cls in comps:
            y = g.loc[g["Class"]==cls, metric_col].dropna().astype(float).values
            if x.size==0 or y.size==0:
                U = np.nan; p = np.nan
            else:
                U, p = mannwhitneyu(x, y, alternative="less", method="auto")
            tmp.append({"Facility": fac, "Class": cls,
                        f"U_{metric_col}": U, f"p_{metric_col}": p})
            pvals.append(p)
        if tmp:
            qvals = bh_correction_vector(np.array(pvals, dtype=float))
            for r, qv in zip(tmp, qvals):
                r[f"q_BH_{metric_col}"] = qv
            rows.extend(tmp)
    return pd.DataFrame(rows)


def format_pq(v):
    if pd.isna(v): return ""
    if v < 1e-4:   return f"{v:.1e}"      # 1.2e-06
    if v < 0.1:    return f"{v:.3f}"      # 0.023
    return f"{v:.2f}"                     # 0.12

def stars_from_q(q):
    if pd.isna(q): return ""
    return "***" if q < 1e-3 else ("**" if q < 1e-2 else ("*" if q < 5e-2 else ""))

def build_table(df_all, metric_col):
    summ  = summarize_counts(df_all, metric_col)
    tests = tests_vs_nilm_one_sided_with_bh(df_all, metric_col)
    out   = summ.merge(tests, on=["Facility","Class"], how="left")


    for col in [f"U_{metric_col}", f"p_{metric_col}", f"q_BH_{metric_col}"]:
        if col in out.columns:
            out.loc[out["Class"]=="NILM", col] = np.nan

    present = [c for c in CLASS_ORDER if c in out["Class"].unique().tolist()]
    rest    = [c for c in out["Class"].unique().tolist() if c not in present]
    out["Class"] = pd.Categorical(out["Class"], categories=present + sorted(rest), ordered=True)
    out = out.sort_values(["Facility","Class"]).reset_index(drop=True)

    out["median [Q1-Q3]"] = out.apply(lambda r: f'{int(r["median"]):,} [{int(r["q1"]):,}-{int(r["q3"]):,}]' if pd.notna(r["median"]) else "", axis=1)
    out["mean ± sd"]       = out.apply(lambda r: f'{r["mean"]:.1f} ± {r["sd"]:.1f}' if pd.notna(r["mean"]) else "", axis=1)
    out["min-max"]         = out.apply(lambda r: f'{int(r["min"]):,}-{int(r["max"]):,}' if pd.notna(r["min"]) else "", axis=1)

    out["p value"]     = out[f"p_{metric_col}"].apply(format_pq)
    out["BH q value"]  = out[f"q_BH_{metric_col}"].apply(format_pq)
    out["significance"] = out[f"q_BH_{metric_col}"].apply(stars_from_q)  # based on BH q

    out["n"] = pd.to_numeric(out["n"], errors="coerce").round().astype("Int64")

    cols = ["Facility","Class","n","median [Q1-Q3]","mean ± sd","min-max", "p value","BH q value","significance"]
    return out[cols]


In [3]:
table_LSIL = build_table(df_all, "LSIL")
table_LSIL.to_csv(f'{fig_dir}STable1_LSIL-statistics.csv', index=False, encoding='utf-8-sig')
table_LSIL

/tmp/ipykernel_154551/3925539613.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_agg).reset_index())


,Facility,Class,n,median [Q1-Q3],mean ± sd,min-max,p value,BH q value,significance
0,1_CIH,NILM,169,1 [0-4],4.3 ± 10.6,0-72,,,
1,1_CIH,ASC-US,72,4 [2-9],7.2 ± 12.0,0-96,1.3e-07,2.1e-07,***
2,1_CIH,LSIL,42,34 [11-82],62.6 ± 85.1,2-437,1.2e-18,6.2e-18,***
3,1_CIH,ASC-H,9,4 [3-9],7.0 ± 7.7,1-26,0.004,0.004,**
4,1_CIH,HSIL,24,18 [10-86],53.5 ± 64.4,1-228,3.7e-11,9.1e-11,***
5,1_CIH,SCC,2,197 [117-276],197.0 ± 224.9,38-356,0.007,0.007,**
6,2_Tsukuba,NILM,55,2 [0-4],2.5 ± 2.7,0-10,,,
7,2_Tsukuba,ASC-US,31,8 [1-15],15.4 ± 31.7,0-177,0.000,0.000,***
8,2_Tsukuba,LSIL,54,50 [21-293],197.8 ± 299.6,"0-1,224",3.1e-16,1.6e-15,***
9,2_Tsukuba,ASC-H,17,3 [1-6],4.2 ± 4.6,0-16,0.099,0.099,


In [4]:
table_HSIL = build_table(df_all, "HSIL")
table_HSIL.to_csv(f'{fig_dir}STable1_HSIL-statistics.csv', index=False, encoding='utf-8-sig')
table_HSIL

/tmp/ipykernel_154551/3925539613.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_agg).reset_index())


,Facility,Class,n,median [Q1-Q3],mean ± sd,min-max,p value,BH q value,significance
0,1_CIH,NILM,169,0 [0-2],2.3 ± 6.0,0-65,,,
1,1_CIH,ASC-US,72,2 [0-4],3.7 ± 5.2,0-28,7.2e-05,0.000,***
2,1_CIH,LSIL,42,3 [1-7],4.7 ± 5.0,0-25,6.3e-07,1.6e-06,***
3,1_CIH,ASC-H,9,6 [2-14],13.9 ± 22.5,0-72,0.000,0.000,***
4,1_CIH,HSIL,24,29 [12-69],61.9 ± 104.3,0-517,1.5e-12,7.4e-12,***
5,1_CIH,SCC,2,340 [187-492],340.0 ± 431.3,35-645,0.004,0.004,**
6,2_Tsukuba,NILM,55,8 [3-24],28.5 ± 59.3,0-355,,,
7,2_Tsukuba,ASC-US,31,19 [7-36],38.7 ± 96.5,0-551,0.044,0.044,*
8,2_Tsukuba,LSIL,54,19 [5-51],41.1 ± 63.3,0-390,0.020,0.025,*
9,2_Tsukuba,ASC-H,17,47 [24-191],101.9 ± 103.9,5-319,3.3e-05,5.6e-05,***
